# Tutorial4: 大模型推理

本节旨在使用 [cpm-bee-2b](https://huggingface.co/openbmb/cpm-bee-2b) 模型展示大模型推理。

分以下几步来实现：
1. 环境安装
2. 下载模型
3. 模型推理

## 1. 环境安装

我们默认用户已经按照 [tutorial_scow_for_ai](../tutorial_scow_for_ai.md) 在 “交互式应用” 中创建了  Jupyter Lab 应用，并已经安装了 conda。现在需要在命令行中创建环境并注册 ipykernal：

```bash
conda create --name cpm-bee python=3.9
conda activate cpm-bee
# 安装内核
conda install ipykernel
# 注册内核
python -m ipykernel install --user --name=cpm-bee --display-name="cpm-bee"
conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia
pip install deepspeed accelerate==0.30.1 transformers==4.33.2
```

（ pytorch 版本需与 cuda 版本对应，请查看版本对应网站：https://pytorch.org/get-started/previous-versions ，通过 nvidia-smi 命令可查看 cuda 版本）

打开本 .ipynb 文件并选择 kernel 为 cpm-bee。硬件资源建议使用1张GPU运行。

CUDA Version: 12.1; Torch Version: 2.3.1

## 2. 下载模型

在联网的命令行中执行以下命令，命令执行位置在当前文件所在的文件夹。

```bash
export HF_ENDPOINT=https://hf-mirror.com # 设置代理，如果可以直接访问则不需要设置代理
huggingface-cli download --resume-download openbmb/cpm-bee-2b --local-dir cpm-bee-2b
```

## 3. 模型推理

CPM-Bee 是一个完全开源、可商用的中英双语基础模型，拥有一百亿参数的容量。我们这里选用的是较小参数的 cpm-bee-2b 模型，下面展示加载 cmp-bee-2b 模型进行中翻英任务并统计生成速度的过程。

推理过程中使用 nvidia-smi 命令可以查看 GPU 运行情况。

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import time

# 模型路径
model_path = "cpm-bee-2b"

# 硬件
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU available, using CPU instead.")

# 分词器
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# 模型
model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    torch_dtype=torch.float16,
    trust_remote_code=True
).cuda()

# 生成
start_time = time.time()
res = model.generate(
    {"input": "NGC 6231是一个位于天蝎座的疏散星团，天球座标为赤经16时54分，赤纬-41度48分，视觉观测大小约45角分，亮度约2.6视星等，距地球5900光年。NGC 6231年龄约为三百二十万年，是一个非常年轻的星团，星团内的最亮星是5等的天蝎座 ζ1星。用双筒望远镜或小型望远镜就能看到个别的行星。NGC 6231在1654年被意大利天文学家乔瓦尼·巴蒂斯特·霍迪尔纳（Giovanni Battista Hodierna）以Luminosae的名字首次纪录在星表中，但是未见记载于夏尔·梅西耶的天体列表和威廉·赫歇尔的深空天体目录。这个天体在1678年被爱德蒙·哈雷（I.7）、1745年被夏西亚科斯（Jean-Phillippe Loys de Cheseaux）（9）、1751年被尼可拉·路易·拉卡伊（II.13）分别再次独立发现。", "prompt": "中翻英", "<ans>": ""},
    tokenizer,
    max_new_tokens=1000,
    min_length=50,
)
end_time = time.time()

# calculate speed
num_generated_tokens = 0
for output in res:
    text = output['<ans>']
    tokens = tokenizer(text, return_tensors='pt')["input_ids"]
    num_tokens = tokens.shape[1]
    num_generated_tokens += num_tokens
time_taken = end_time - start_time
tokens_per_second = num_generated_tokens / time_taken
print(f"Generated {num_generated_tokens} tokens in {time_taken:.2f} seconds ({tokens_per_second:.2f} tokens/second)")

print(res[0]['<ans>'])

---

> 作者: 黎颖; 龙汀汀
>
> 联系方式: yingliclaire@pku.edu.cn;   l.tingting@pku.edu.cn